# Using Python and Jupyter for Data Analysis

Welcome to the exciting world of Data Science! The environment you are in right now is what many professional Data Scientists are using to explore and analyse complex data sets. It is called Jupyter and we are using the environment with a programming language called Python. 

Jupyter is providing a whole ecosystem of tools, what we are specifically using here is a Jupyter Notebook. 

A Jupyter Notebook is a completely interactive, web-based environment in which you can use the programming language, in our case Python, in your browser. 

Usually you would use your own computer, but for this class we are using Jupyter in the cloud, more specifically we are using a service called Binder, which allows to build a stand-alone Jupyter server just for your notebooks and share them with others. There are many things going on behind the scenes to make this happen, but Binder itself is building the dedicated Jupyter server and the running it on the Google Cloud. 

## Let's get started

A notebook consists of a vertical column of cells like the one this text is in. Cells are like paragraphs in a text, but they are 'active', can be executed and edited much more like cells in an Excel spreadsheet. Like in a spreadsheet you can move from one cell to the next using the cursor keys or the mouse and the green or blue box around the current cell indicates where you are. Easy!

There are two main kinds of cells, one is called 'Markdown', the other are Code cells, in our case for Python code. Markdown is for text. You can enter a cell by pressing 'Enter' or double clicking. You can exit a cell by pressing Esc. Once in a cell you can change the content, like in Excel. If you want to execute the cell you can press Shift+Enter (execute and go to next) or Control+Enter (just execute).

Let's try a few simple code cells (remember that you have the whole Python language at your fingertips!):

In [ ]:
x = 14 * 3

In [ ]:
y = 14**3 
print('x = {0}; y = {1}'.format(x, y))

In [ ]:
x < y

If you want to learn Python, there are myriads of resources available on the web...

## Now let's get to the data analysis:

### First we need to load a powerful library:
Libraries are software packages written for Python for specific purposes.

In [ ]:
import pandas as pd               # This is the workhorse for everything Data Science in Python

#### Read the data using Pandas:

In [ ]:
df = pd.read_csv('data/data1000.csv')  # That's just the CSV data you've received

The command above does not produce any output, but the result of reading the file is now contained in the variable *df*.

#### How does the data look like?

Just typing the variable name shows a summary of the content.

In [ ]:
df

As you can see the variable *'df'* contains something quite different from your usual Math variable! It contains the whole data set in a quite convenient form to deal with in a programming language.

You can get the headings of the columns by 'asking' the variable:

In [ ]:
df.keys()

There is a pretty strange column in there called *'Unnamed: 20'*. Let's examine what it contains:

In [ ]:
df['Unnamed: 20']

Seems to contain just invalid numbers (NaN means *'Not a Number'*), but let's verify that:

In [ ]:
df['Unnamed: 20'].notnull().values.any()

In words the line above means: Are there *any values* in the column *'Unnamed: 20'* that are *not null*?
The answer is *False*, which means there are no such values.

As you can see there are functions for all kinds of use cases in Pandas...

Also to answer the exactly opposite question:

In [ ]:
df['Unnamed: 20'].isnull().values.all()

Note the usage of *all()* here!

#### Create a histogram plot of reaction times of females and males

Now we will try to answer the question whether there is a significant difference between the measured reaction times of female and male students. 

To do this, we need to get the values separated by the *Gender* column. You can do this the hard way, but of course there is also a Pandas function to perform exactly that for you and even for the complete data set at once! This function is called *groupby* and takes the column name you want to use for grouping as an argument.

Calling it naîvely like this, results in a pretty strange output:

In [ ]:
df.groupby('Gender')

This is plain Python gibberish, but it reveals that the result is actually something that is called an *object*. Very often objects have quite a lot of functionality embedded in it. You can examine this by using the Python built-in function *dir*.

In [ ]:
dir(df.groupby('Gender'))

In [ ]:
df.groupby('Gender').plot()

In [ ]:
# Some global settings used a lot below
GROUPBY = ['Favourite_physical_activity']   # dataset will be grouped by these columns
VALUE_COLUMN = 'Ageyears'   # the column used as a value (should be numerical)
X_TITLE = VALUE_COLUMN           # the title for the x-axis
Y_TITLE = GROUPBY                # the title for the y-axis
SCALED = True                    # normalized histograms or not

In [ ]:
df_G = df.groupby(GROUPBY)       # perform the grouping

In [ ]:
df_G.groups.keys()

In [ ]:
# This is pretty powerful: it plots the histograms for all groups using a single command. The rest of the keywords are just to tidy things up a bit.

ax = df_G[VALUE_COLUMN].plot.hist(figsize=(12,8), legend=True, alpha=0.5, density=SCALED)
_ = ax[-1].set_xlabel(X_TITLE)
_ = ax[-1].set_ylabel(Y_TITLE)

### Now we are trying to deal with the 'outliers':

We need to do this for the groups separately:

In [ ]:
# This is assuming that we have just two groups:
df_f = df[VALUE_COLUMN].where(df['Gender'] == 'F')
df_m = df[VALUE_COLUMN].where(df['Gender'] == 'M')

Since we will use this a few times, we just define a couple of functions and use another library to help us with that job.

In [ ]:
import scipy, scipy.stats                      # a big Python module for everything science data exploration

In [ ]:
def remove_outliers(s, low=None, high=None, index=False):
    """
    Helper function to remove outliers above or below certain quantiles
    """
    if not low:
        low = 0.25   # standard for outlier definition
    if not high:
        high = 1 - low
    iqr = scipy.stats.iqr(s, nan_policy='omit')
    if not index:
        return s[s.between(s.quantile(low) - 1.5 * iqr, s.quantile(high) + 1.5 * iqr)]
    else:
        return s.between(s.quantile(low) - 1.5 * iqr, s.quantile(high) + 1.5 * iqr)

In [ ]:
def outlier_cutoff(s):  
    """
    Helper function to get the IQR and standard quantile bounds.
    """
    iqr = scipy.stats.iqr(s, nan_policy='omit')
    return {'iqr':iqr, 'lower bound': s.quantile(0.25)-1.5*iqr, 'upper bound': s.quantile(0.75)+1.5*iqr}

In [ ]:
df_fc = remove_outliers(df_f, low=0.25)
df_mc = remove_outliers(df_m, low=0.25)
df_f = df[VALUE_COLUMN].where(df['Gender'] == 'F')

In [ ]:
import matplotlib.pyplot as plt   # very powerful python plotting library

### we can plot without even knowing the number or name of the groups

In [ ]:
# get the actual group names
group_names = list(df_G.groups.keys()) 

quant = 0.25  # quantile to remove outliers 

fig = plt.figure(figsize=[12,8])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes

# remove outliers in first group
G0_cleaned = remove_outliers(df_G.get_group(group_names[0])[VALUE_COLUMN], low=quant)

# do the plot for first plot
G0_cleaned.plot.hist(ax=ax, figsize=(12,8), legend=True, alpha=0.5, density=SCALED)

# create a subplots for all remaining groups
for g in group_names[1:]:  
    # sometimes the group keys have NaN values: Not good, because histogram will fail!
    # we will just omit that subplot.
    try: 
        g_cleaned = remove_outliers(df_G.get_group(g)[VALUE_COLUMN], low=quant)
        g_cleaned.plot.hist(ax=ax, figsize=(12,8), legend=True, alpha=0.5, density=SCALED)
    except: 
        next

# put the group names in the legend and set the axis labels
_ = ax.legend(group_names)
_ = ax.set_xlabel(X_TITLE)
_ = ax.set_ylabel('Frequency')

# Change y axis label if the histogram is normalized.
if SCALED: ax.set_ylabel('Normalized Frequency')

Let's do something a bit more exciting, also showing the power of matplotlib.

In [ ]:
fig = plt.figure(figsize=[12,8])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes

_ = df.boxplot(ax=ax, by=GROUPBY, column=VALUE_COLUMN, vert=False)
_ = ax.set_xlabel(X_TITLE)
_ = ax.set_ylabel(Y_TITLE)
_ = ax.set_title('')

There is an interesting kind of plot, which shows something similar to the boxplot above, but even more additional information. That kind of plot is called a *violin plot* and is not available in matplotlib. Thus we load yet another library to enable this.

In [ ]:
import seaborn as sns             # high level plots

The library allows a very flexible configuration of the plots and thus the commands look a bit daunting. The way this is implemented below also allows for more than two groups to be shown correctly (think about more groups than just female and male). We will explore this afterwards.

In [ ]:
fig = plt.figure(figsize=[12,8])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes

dfc = df
# dfc = df.loc[df.Reaction_time <= 1.5]

if len(GROUPBY) == 2:
    _ = sns.violinplot(data=dfc, x=GROUPBY[0], y=VALUE_COLUMN, hue=GROUPBY[1], split=True, inner="quartile", bw=0.25, cut=0)
else:
    dfc['dummy'] = 'A'
    _ = sns.violinplot(data=dfc, x='dummy', y=VALUE_COLUMN, hue=GROUPBY[0], width=0.25, split=False, inner="quartile", bw=0.25, cut=0)
    _ = ax.set_xlabel(X_TITLE)
    _ = ax.set_xticklabels('')
_ = ax.set_ylabel(Y_TITLE)
_ = ax.set_title('')

The plot shows many things in one go:

1. Both groups side-by-side (female blue, male orange)
2. The smoothed distributions
3. Median, lower and upper quartiles
4. The tails as well.

... and it even looks quite pleasing, which is an important aspect for plots.

## Examine different hypothesis:
Arm span of a person is approximately the same as the person's height.

That would mean that they values plotted against each other should approximately follow a line with a gradient of 1. 

Remember the equation y = m * x + c ?? 

In [ ]:
df.keys()

In [ ]:
X_COLUMN = 'Height'
Y_COLUMN = 'Arm_Span'

In [ ]:
fig = plt.figure(figsize=[12,8])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes
# ax.set_xlim([100, 230])
# ax.set_ylim([100, 230])
ax = sns.regplot(ax=ax, x=df[X_COLUMN],y=df[Y_COLUMN])

In [ ]:
fig = plt.figure(figsize=[12,2])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes
_ = sns.residplot(x=df[X_COLUMN],y=df[Y_COLUMN])

### Same but using a feature rich stats library

In [ ]:
import statsmodels.api as sm      # a Python statistics module 

In [ ]:
X = sm.add_constant(df[X_COLUMN])  # make sure the algorithm has enough degrees of freedom
Y = df[Y_COLUMN]                 # use Arm_Span; Hypothesis: Arm_Span ~= Height
model = sm.OLS(Y, X).fit()         # Perform the fit
print(model.summary())             # show fitting summary

### R-squared is 0.508, a reasonable fit, but the data has quite a few outliers

### Cleanup outliers:

In [ ]:
out = model.outlier_test()     # check for outliers

select = abs(out['student_resid']) <= 2.  

yfit_df = pd.DataFrame(model.fittedvalues)  # the fitted values (on the line)

Xdf = pd.DataFrame(X.values[:,1])
Ydf = pd.DataFrame(Y)
Xclean = Xdf.loc[(select).values].values # remove all X coords with a residual > 2
Yclean = Ydf.loc[(select).values].values # remove all Y coords with a residual > 2
yfit_clean = yfit_df.loc[(select).values].values

print("{0} outliers identified:".format(len(df[X_COLUMN]) - len(Xclean)))

# perform the fit without outliers
Xfit = sm.add_constant(Xclean)
cfit = sm.OLS(Yclean, Xfit).fit()
print("Fit results without outliers:")
print(cfit.summary())

### R-squared is now 0.791: quite an improvement!

In [ ]:
fig = plt.figure(figsize=[12,8])  # start a new figure with a certain size
ax = fig.gca()                    # get the figure axes

_ = plt.scatter(df[X_COLUMN], Y)                                # points with outliers
_ = plt.plot(df[X_COLUMN], model.fittedvalues, color="red")     # fit line with outliers
_ = plt.scatter(Xclean, Yclean, color="orange")                 # points without outliers
_ = plt.plot(Xclean, cfit.fittedvalues, color="green")          # fit line without outliers
ax.set_xlabel(X_COLUMN)
ax.set_ylabel(Y_COLUMN)
plt.show()

In [ ]:
print(outlier_cutoff(df_f))